In [ ]:
!sudo apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl
!python3 -m pip install --upgrade pip
!python3 -m pip install konlpy
!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.20).
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
g++ set to manually installed.
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless python-pip-whl python3-asn1crypto
  python3-cffi-backend python3-crypto python3-cryptography python3-idna
  python3-keyring python3-keyrings.alt python3-pkg-resources
  python3-secretstorage python3-setuptools python3-six python3-wheel
  p

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

In [ ]:
sample = pd.read_csv('./kerc_datasets/example_submission.csv')
train_data = pd.read_csv('./KERC22Dataset_PublicTest/train_data.tsv', sep='\t')
train_label = pd.read_csv('./KERC22Dataset_PublicTest/train_labels.csv')
test_data = pd.read_csv('/content/KERC22Dataset_PublicTest/public_test_data.tsv', sep='\t')

In [ ]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [ ]:
sample.head()

,Id,Predicted
0,468,dysphoria
1,11743,dysphoria
2,19,dysphoria
3,1070,euphoria
4,1903,dysphoria


In [ ]:
train_data

,sentence_id,person,sentence,scene,context
0,1,어영,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,S0104,NaN
1,2,어영,우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,S0108,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는"
2,3,지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
3,4,순경,근무중인데 어딜가?,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
4,5,지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
...,...,...,...,...,...
7334,12285,청난,종남이 맛있는것도 사주고 잘 챙겨줘.,S3576,NaN
7335,12286,건강,걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.,S3576,NaN
7336,12287,종남,아빠. 우리 저거 타.,S3576,NaN
7337,12288,건강,"나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너도 내몸땡이에 ...",S3576,NaN


In [ ]:
test_data

,sentence_id,person,sentence,scene,context
0,10,현찰,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람..."
1,11,현찰,"계약할 서류는 태연희실장 통해서 보내 드렸는데 확인해 보셨죠? 예, 예.",S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람..."
2,12,경위1,김순경 경사님 사모님 아니십니까?,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람..."
3,13,과자,아 예.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람..."
4,14,경위1,저 기억하시죠? 00지구대에서 김경사님 모시고 근무했었습니다.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람..."
...,...,...,...,...,...
2561,12256,우미현찰애들,다녀오겠습니다.,S3570,가방든 현찰우미. 애들 신났고. 순경건강 나와선. 부러운듯 보는 종남
2562,12257,건강,부럽냐?,S3570,우르르 몰려 나가는. 종남 부러운듯 보는. 순경은 들어갔고. 종남 보는 건강
2563,12258,종남,응.,S3570,우르르 몰려 나가는. 종남 부러운듯 보는. 순경은 들어갔고. 종남 보는 건강
2564,12259,건강,그럼 까짓거 우리도 놀러가지뭐.,S3570,우르르 몰려 나가는. 종남 부러운듯 보는. 순경은 들어갔고. 종남 보는 건강


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7339 entries, 0 to 7338
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sentence_id  7339 non-null   int64 
 1   person       7339 non-null   object
 2   sentence     7339 non-null   object
 3   scene        7339 non-null   object
 4   context      5322 non-null   object
dtypes: int64(1), object(4)
memory usage: 286.8+ KB


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2566 entries, 0 to 2565
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sentence_id  2566 non-null   int64 
 1   person       2566 non-null   object
 2   sentence     2566 non-null   object
 3   scene        2566 non-null   object
 4   context      1795 non-null   object
dtypes: int64(1), object(4)
memory usage: 100.4+ KB


In [ ]:
len(train_data['person'].unique())

87

In [ ]:
len(test_data['person'].unique())

59

In [ ]:
len(intersection(train_data['person'].unique(),test_data['person'].unique()))

39

In [ ]:
train_data['label'] = train_label['label']

In [ ]:
train_data

,sentence_id,person,sentence,scene,context,label
0,1,어영,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,S0104,NaN,dysphoria
1,2,어영,우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,S0108,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는",dysphoria
2,3,지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,neutral
3,4,순경,근무중인데 어딜가?,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,dysphoria
4,5,지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,euphoria
...,...,...,...,...,...,...
7334,12285,청난,종남이 맛있는것도 사주고 잘 챙겨줘.,S3576,NaN,euphoria
7335,12286,건강,걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.,S3576,NaN,euphoria
7336,12287,종남,아빠. 우리 저거 타.,S3576,NaN,euphoria
7337,12288,건강,"나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너도 내몸땡이에 ...",S3576,NaN,euphoria


# Get-Started

In [ ]:
from konlpy import tag
from konlpy.utils import pprint

In [ ]:
tokenizer = tag.Mecab()
print('|'.join(tokenizer.morphs(train_data['sentence'][1])))

우리|아무래도|안|되|겠|다|.|이게|최선|인|거|같|애|.|평생|잊|지|않|을게|.|행복|하|길|바란다|.


In [ ]:
def processtext(x):
  result = tokenizer.morphs(x)
  result = [x for x in result if x != '.']
  return result

In [ ]:
train_data

,sentence_id,person,sentence,scene,context,label
0,1,어영,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,S0104,NaN,dysphoria
1,2,어영,우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,S0108,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는",dysphoria
2,3,지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,neutral
3,4,순경,근무중인데 어딜가?,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,dysphoria
4,5,지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,euphoria
...,...,...,...,...,...,...
7334,12285,청난,종남이 맛있는것도 사주고 잘 챙겨줘.,S3576,NaN,euphoria
7335,12286,건강,걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.,S3576,NaN,euphoria
7336,12287,종남,아빠. 우리 저거 타.,S3576,NaN,euphoria
7337,12288,건강,"나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너도 내몸땡이에 ...",S3576,NaN,euphoria


In [ ]:
train_data.label.value_counts() / train_data.shape[0]

dysphoria    0.616705
euphoria     0.268020
neutral      0.115275
Name: label, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(train_data, test_size=0.15, random_state=69420)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)

In [ ]:
dictation = np.append(train_df['sentence'].values, train_df['context'].dropna().values)

In [ ]:
vtr = TfidfVectorizer(tokenizer=processtext, ngram_range=(1,2), min_df=20, sublinear_tf=True)
vtr_learn = vtr.fit(dictation)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


In [ ]:
valid_df

,sentence_id,person,sentence,scene,context,label
0,10922,이상,"집에 갔습니다. 장인어른, 저 배고픈데 밥좀 사주십시요.",S3212,"착잡한 범인. 사정하는 솔이 모습 떠올리고, 안좋은 범인. 노크소리에 이어 들오는 이상",dysphoria
1,10072,이상,큰형수님인데 집에 빨리 오래.,S2937,NaN,euphoria
2,8040,청난,동서가 그랬잖아! 아유 답답해. 돌아 버리겠네.,S2412,둘러앉은 순경과자건강현찰. 울먹대는 청난,dysphoria
3,734,이상,깨셨어요? 아직 삼거리전인데.,S0309,NaN,neutral
4,3630,성미,후식까지 챙기셨네요. 우리 왕검사 딸기맛 안좋아하는데.,S1201,"탁자로 가서, 감정 안들킬려고 애쓰며 쓸어담는 어영. 손 덜덜 떨리고. 좀 떨어져 ...",dysphoria
...,...,...,...,...,...,...
1096,73,우미,나중에 언제? 여보세요?,S0124,NaN,dysphoria
1097,121,현찰,결혼해서 지금까지 나도 할만큼 했어. 다신 처갓집 얘기 내앞에서 꺼내지도 마.,S0149,"들어오는 우미, 애들방앞에 걸터앉는데",dysphoria
1098,5849,이상,그럼. 큰형수 나왔다는 이여대 나왔어. 일본 유학도 갔다왔구.,S1807,"소개받을때마다 어영은 고개 숙이며 인사하고, 식구들은 전부 어영을 살피듯 눈떼지않고...",euphoria
1099,8331,과자,아무래도 너네 분가해야겠다.,S2454,건강 옷입는데,dysphoria


In [ ]:
train_tran_sentence = vtr_learn.transform(train_df['sentence'].astype('U'))
train_tran_context = vtr_learn.transform(train_df['context'].astype('U'))
train_tran_sentence_array = [i.toarray()[0] for i in train_tran_sentence]
train_tran_context_array = [i.toarray()[0] for i in train_tran_context]

In [ ]:
valid_tran_sentence = vtr_learn.transform(valid_df['sentence'].astype('U'))
valid_tran_context = vtr_learn.transform(valid_df['context'].astype('U'))
valid_tran_sentence_array = [i.toarray()[0] for i in valid_tran_sentence]
valid_tran_context_array = [i.toarray()[0] for i in valid_tran_context]

In [ ]:
# np.append(valid_tran_sentence_array[0],valid_tran_context_array[0]).shape

In [ ]:
xtrain = []
for i,v in enumerate(train_tran_sentence_array):
  xtrain.append(np.append(train_tran_sentence_array[i],train_tran_context_array[i]))

In [ ]:
xvalid = []
for i,v in enumerate(valid_tran_sentence_array):
  xvalid.append(np.append(valid_tran_sentence_array[i],valid_tran_context_array[i]))

In [ ]:
ytrain = train_df['label']
yvalid = valid_df['label']

In [ ]:
base = LogisticRegression(C=3., penalty='l2', solver='liblinear', dual=False, multi_class='ovr')
base.fit(xtrain, ytrain)
base.score(xvalid, yvalid)

0.7556766575840145

In [ ]:
prediction = base.predict(xvalid)
prob = base.predict_proba(xvalid)

# Test

In [ ]:
test_data

In [ ]:
test_tran_sentence = vtr_learn.transform(test_data['sentence'].astype('U'))
test_tran_context = vtr_learn.transform(test_data['context'].astype('U'))
test_tran_sentence_array = [i.toarray()[0] for i in test_tran_sentence]
test_tran_context_array = [i.toarray()[0] for i in test_tran_context]

In [ ]:
xtest = []
for i,v in enumerate(test_tran_sentence_array):
  xtest.append(np.append(test_tran_sentence_array[i],test_tran_context_array[i]))

In [ ]:
len(xtest)

In [ ]:
len(xtest[0])

In [ ]:
pred = base.predict(xtest)

In [ ]:
result = pd.DataFrame(columns=['Id', 'Predicted'])

In [ ]:
result['Id'] = test_data['sentence_id']
result['Predicted'] = pred

In [ ]:
result.Predicted.value_counts() / test_data.shape[0]

In [ ]:
result.to_csv('./submission.csv', index=False)

In [ ]:
dfnew = pd.read_csv('/content/submission.csv')

In [ ]:
dfnew.Predicted.value_counts() / test_data.shape[0]